# Writing SHA-256 in Python

SHA-256 has become an incredibly useful cryptographic function. It has the capability to convert any data into a unique 32-byte string (256 bits respectively). The real magic comes from its following two properties.

1. Any change in the incoming data will produce a completely new and unique 32-byte string (if two pieces of data can produce the same string it is called a collision and can be an enormous security problem)
2. The original data cannot be reproduced from the hash, therefore the function is one directional. This means the hash can be shared freely without any fear of the original data being 

<img class="image" src="{{ site.url }}/assets/images/sha256/diagram.png" alt="SHA-256 Basics">

How exactly does it do this? Well, lets make our own SHA-256 function to find out!

This is pretty easy to start, we just need a function to take in any data that can be converted to binary. In this example we'll limit our data to string values for representation sake. With this we can easily convert teach value to it's unicode code point by using Python's `ord()` function and convert this integer to binary by using Python 3's format function.

In [1]:
def _str_to_bin(data_string):
    """Returns the binary representation of a string using unicode (or ASCII)
       representation of the string values

    args:
        data_string (str): Incoming string in unicode/ASCII format

    return args:
        binary_data (str): Binary representation of the string
    """
    print("String values:", list(data_string))
    unicode_points = [ord(char) for char in data_string]
    print("Unicode points:", unicode_points)
    binary_values = ['{0:08b}'.format(point) for point in unicode_points]
    print("Binary values:", binary_values)
    binary_data = ''.join(binary_values)
    print('Binary data: ', binary_data)
    return binary_data

binary_data = _str_to_bin('abc')

String values: ['a', 'b', 'c']
Unicode points: [97, 98, 99]
Binary values: ['01100001', '01100010', '01100011']
Binary data:  011000010110001001100011


Now that we have our binary data we need to preprocess it to have it in the correct format for the future SHA-256 functions. This is is achieved with the following three properties.

1. A 1 is added to the end of the binary data.
2. The length of the binary data (3 x 8 = 24) is appended to the end of the binary blob in a 64 bit number.
3. The final 1 and the 64 bit number from 1. and 2. are separated by 0s so that the total length of the binary blob is a multiple of 512.

<img class="image" src="{{ site.url }}/assets/images/sha256/diagram.png" alt="SHA-256 Basics">

The purpose of this step is to have 512 bit blocks which will be used for each step of the SHA-256 process. Furthermore, this 512 bit block will be used in 16 smaller 32 bit blocks for each sub-process of each step. Therefore, for the data to be easily used, and in binary format it is also converted back to int 512 bit tuples each with 16 32-bit integers.

In [22]:
def preprocess_data(binary_data):
    print("Binary data:", binary_data)
    data_length = '{0:064b}'.format(len(binary_data))
    print("Length:", len(binary_data), "bits")
    print(len(binary_data), "in 64 bit binary format:", data_length)
    padding = '0' * (512 - (len(binary_data) + 1 + 64) % 512)
    print("Padding needed for a 512 bit block:", len(padding), "0s")
    binary_string = binary_data + '1' + padding + data_length
    print("Final binary string:", binary_string)
    processed_values = [int(binary_string[i:i + 32], 2)
                        for i in range(0, len(binary_string), 32)]
    output_data = [tuple(processed_values[i:i + 16]) 
                   for i in range(0, len(processed_values), 16)]
    print("Final data ints for processing: ", output_data)
    return output_data

preprocessed_data = preprocess_data(binary_data)

Binary data: 011000010110001001100011
Length: 24 bits
24 in 64 bit binary format: 0000000000000000000000000000000000000000000000000000000000011000
Padding needed for a 512 bit block: 423 0s
Final binary string: 01100001011000100110001110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011000
Final data ints for processing:  [(1633837952, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24)]


So far we haven't touched anything to do with the SHA-256 process. The power of SHA-256 comes from two main factors:

1. Completely random seed values
2. A number of bitwise operations (operations which move 1s and 0s around in a specific manor)

The bitwise operations used in SHA-256 are as follows.

***Rotate right operation (ROTR)***<br>
Rotates the binary data right n units, for example:<br>
ROTR(100010, 1) --> 010001<br>
ROTR(100010, 2) --> 101000

***Shift right operation (SHR)***<br>
Same as ROTR, however, data on the right side is lost and the left side is filled with 0s<br>
SHR(100010, 1) --> 010001<br>
SHR(100010, 2) --> 001000

***Choose operation (Ch)***<br>
Takes 3 equal length binary values, x, y, and z. For each binary value the operation checks if x is 1 or 0. If x is 1, the value is chosen from y and if x is 0 the value is chosen from z.

<pre>
```

| x      | 1010 |
| y      | 0011 |
| z      | 1100 |
| output | 0110 |

```
</pre>

***Majority operation (Maj)***<br>
Takes 3 equal length binary values, x, y, and z. For each binary value the operation checks the total numbers of 1s and 0s in x, y and z. If a majority of the values are 1 the output is 1 for that position and if the majority are 0s the output is 0.

<pre>
```

| x      | 1010 |
| y      | 0011 |
| z      | 1100 |
| output | 1010 |

```
</pre>